In [1]:
import matplotlib as plt
import networkx as nx
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
import os
import wget
from time import sleep
import pandas as pd
import random
import numpy
import csv

### Este código se mete a los followers de gabi, luego se mete a cada follow y saca los nombres de quienes también siguen esa cuenta. La idea es ir buscando mutualFollows para enfocarse en ellos y encontrar targets con más facilidad.

In [2]:
c = webdriver.ChromeOptions()
c.add_argument("--incognito")
# c.add_argument("--start-maximized")
driver = webdriver.Chrome(' ', options=c) ## Input driver directory
driver.get('https://www.instagram.com/')
username = WebDriverWait(driver,10).until(
    EC.element_to_be_clickable((By.XPATH, '//*[@id="loginForm"]/div/div[1]/div/label/input'))
    )

password = WebDriverWait(driver,10).until(
    EC.element_to_be_clickable((By.XPATH, '//*[@id="loginForm"]/div/div[2]/div/label/input'))
    )

username.send_keys(' ') ## Input username
password.send_keys(' ') ## Input Password

log_in = WebDriverWait(driver,10).until(
    EC.element_to_be_clickable((By.XPATH, '//*[@id="loginForm"]/div/div[3]/button/div'))).click()
    
try:
    not_now = WebDriverWait(driver,10).until(EC.element_to_be_clickable((By.XPATH, "//button[contains(text(),'Not Now')]"))).click()
    not_now2 = WebDriverWait(driver,10).until(EC.element_to_be_clickable((By.XPATH, "//button[contains(text(),'Not Now')]"))).click()
except:
    pass

C:\Users\ale45\AppData\Local\Temp\ipykernel_5304\3431399881.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('C:/Users/ale45/Desktop/WebScrapping/chromedriver.exe', options=c)


In [3]:
def get_following_and_followers_num(User):
    link = 'https://www.instagram.com/' + User
    driver.get(link)
    sleep(3)
    num_fwing = driver.find_element(By.XPATH,'/html/body/div[2]/div/div/div/div[1]/div/div/div/div[1]/div[1]' + 
        '/div[2]/section/main/div/header/section/ul/li[3]/a/div/span/span').get_attribute('innerHTML')
    num_fwers = driver.find_element(By.XPATH,'/html/body/div[2]/div/div/div/div[1]/div/div/div/div[1]/div[1]' +
        '/div[2]/section/main/div/header/section/ul/li[2]/a/div/span/span').get_attribute('innerHTML')
    return int(num_fwing), int(num_fwers)


def scroll_em(num_seg):
    for i in range(int(num_seg/12) + 2):
        driver.execute_script("""
        var scrollt = document.querySelector('div[role="dialog"] ._aano')
        scrollt.scrollTop = scrollt.scrollHeight
        """)
        sleep(3)



def get_followers_names(User, num_fwers):
    driver.get('https://www.instagram.com/' + User + '/followers/')
    sleep(3)
    scroll_em(num_fwers)
    names = []
    xpath = ('//*[@class=" _ab8y  _ab94 _ab97 _ab9f _ab9k _ab9p _abcm"]')
    names_driver = driver.find_elements(By.XPATH,xpath)
    names = [name.get_attribute('innerHTML') for name in names_driver]

        
    return names

In [4]:
def main (Set):
    Relations = pd.DataFrame()
    for i in Set:
        driver.get('https://www.instagram.com/' + i)
        sleep(2.5)
        try:
            more_follows = driver.find_element(By.XPATH,'//*[@class="_aacl _aacn _aacv _aacy _aad6"]'
                ).get_attribute('innerHTML')
        except:
            Relations[i] = [0] * len(Relations)
            continue

        if ('more' in more_follows):
            names = []
            number = more_follows.split('>')
            number = number[-1].split(' ')
            number = int(number[2])
            click_more = WebDriverWait(driver,10).until(EC.element_to_be_clickable(
                    (By.XPATH, ('//div[contains(text(),"Followed by")]')))).click()
            click_link = WebDriverWait(driver,10).until(EC.element_to_be_clickable(
                    (By.XPATH, '//a[contains(text(),"See all followers")]'))).click()
            scroll_em(number+3)
            Botons_num  = driver.find_elements(By.XPATH,"//div[contains(text(),'Following')]")
            i1 = 1
            while i1 <= len(Botons_num)-1:
                xpath_name = ('/html/body/div[2]/div/div/div/div[2]/div/div/div[1]/div/div[2]' +
                    '/div/div/div/div/div[2]/div/div/div[2]/div[1]/div/div[' + str(i1) + ']/div[2]/div[1]/div/div/div/a/span/div')
                name = driver.find_element(By.XPATH,xpath_name).get_attribute('innerHTML')
                names.append(name)
                i1 += 1 

        else:
            names = driver.find_elements(By.XPATH,'//*[@class="_aaai"]')
            names = [j.get_attribute('innerHTML') for j in names]
        Relations[i] = [0] * len(Relations)
        for k in names:
            Relations.at[k,i] = 1
    
    Relations = Relations.fillna(0)
    return (Relations)

In [5]:
Followers_gabi_num = get_following_and_followers_num('gabal_2003')[1]
Followers_gabi = get_followers_names('gabal_2003',Followers_gabi_num)
Followers_gabi = Followers_gabi[0:Followers_gabi_num]


In [12]:
set_sz = int(Followers_gabi_num/5)
set_A = Followers_gabi[:set_sz]
set_B = Followers_gabi[set_sz:2*set_sz]
set_C = Followers_gabi[2*set_sz:3*set_sz]
set_D = Followers_gabi[3*set_sz:4*set_sz]
set_E = Followers_gabi[4*set_sz:]
Set_list = [set_A,set_B,set_C,set_D, set_E]

In [10]:
Relations_complete = pd.DataFrame()
cont = 0
for set_it in Set_list:
    Relations_main = main (set_it)
    Relations_main.to_csv('Relations' + str(cont) + '.csv')
    Relations_complete = pd.concat([Relations_complete, Relations_main], axis=1)
    cont = cont + 1

KeyboardInterrupt: 

In [14]:
Relations_complete = Relations_complete.fillna(0)
Relations_complete.to_csv('Relations_updt.csv')